In [1]:
#| default_exp pipeline

In [2]:
#| exporti
import torch
from whisperspeech.t2s_up import TSARTransformer
from whisperspeech.s2a_delar_mup import SADelARTransformer
from whisperspeech.a2wav import Vocoder

In [3]:
#| export
class Pipeline:
    def __init__(self, use_kv_cache=True):
        self.t2s = TSARTransformer.load_model(use_kv_cache=use_kv_cache).cuda()
        self.s2a = SADelARTransformer.load_model(use_kv_cache=use_kv_cache).cuda()
        self.vocoder = Vocoder()

    def generate_atoks(self, text, speaker="3645"):
        text = text.replace("\n", " ")
        stoks = self.t2s.generate(text, T=.5, top_k=3)
        atoks = self.s2a.generate(stoks, [speaker], T=2, top_k=8)
        return atoks
        
    def generate(self, text, speaker="3645"):
        return self.vocoder.decode(self.generate_atoks(text, speaker))

    def generate_to_file(self, fname, text, speaker="3645"):
        self.vocoder.decode_to_file(fname, self.generate_atoks(text, speaker))

    def generate_to_notebook(self, text, speaker="3645"):
        self.vocoder.decode_to_notebook(self.generate_atoks(text, speaker))